In [1]:
import ctypes

In [2]:
#i = c848.C848_InterfaceSetupDlg()

In [3]:
def load_drivers(filename='C848_DLL.dll'):
    return ctypes.CDLL('C848_DLL.dll')

In [4]:
def conncect_to_controller(driver):
    '''
    connects to C848 controler using given driver
    return controller id or None if conection was not successful
    '''
    connect_fun = driver.C848_ConnectRS232
    connect_fun.argtypes = [ctypes.c_int, ctypes.c_long]
    controller_id = connect_fun(4, 57600)
    return controller_id if controller_id != -1 else None

In [5]:
def is_connected(controller_id):
    '''
    checks if connection to C848 was established
    return True if there connection a connection to controller with given controller_id, else returns false
    '''
    return bool(c848.C848_IsConnected(ctypes.c_int(controller_id)))  

In [6]:
def close_connection(controller_id):
    '''
    closes connection to C848 controler with given controller_id
    ''' 
    c848.C848_CloseConnection(ctypes.c_int(controller_id))

In [7]:
def get_axes(axes = 'XYZ'):
    '''
    converts "xyz" string to "ABC" string needed to give axes to c848 controller
    #A = x
    #B = y
    #C = z
    '''
    axes_map = {'x': 'A', 'y': 'B', 'z': 'C'}
    abc_list = [axes_map[ax] for ax in list(axes.lower())]
    return ''.join(abc_list)

In [8]:
def get_szAxes(axes = 'XYZ'):
    '''
    Takes string defining axes as parameter (axes defined in XYZ string)
    return char pointer to string with axes, usually char *const szAxes parameter in c848 dll functions.
    '''
    axes_ABC = get_axes(axes)
    return ctypes.c_char_p(axes_ABC.encode('utf-8'))

In [9]:
def convert_id(controller_id):
    '''
    converts controller_id to c_int
    '''
    return ctypes.c_int(controller_id)

In [10]:
def create_bool_array(size = 1, values=0):
    '''
    takes size of array as parameter
    creates bool array of given size filled with true values
    returns c pointer to this array
    '''
    b = (ctypes.c_bool * size)(*[values] * size)
    return ctypes.cast(b, ctypes.POINTER(ctypes.c_bool))

In [11]:
def create_double_array(size = 1, positions=None):
    '''
    takes size of array as parameter
    creates double array of given size filled with 25.0 values
    returns c pointer to this array
    '''
    if positions == None:
        positions = [25.0] * size
        
    arr = (ctypes.c_double * size)(*positions)
    return ctypes.cast(arr, ctypes.POINTER(ctypes.c_double))

In [12]:
def get_axes_positions(controller_id, axes='xyz'):
    '''
    function gives positions of given axes
    
    returns list of positions in order of given axes
    '''
    c_id = convert_id(controller_id)
    sz_axes = get_szAxes(axes)
    c_double_array = create_double_array(size=len(axes))
    if c848.C848_qPOS(c_id, sz_axes, c_double_array):
        return c_double_array[:len(axes)]
    else:
        print('something went terribly wrong')

In [13]:
def move_axes_to_abs(controller_id, axes='xyz', positions = [25.0, 25.0, 25.0]):
    '''
    moves axes to absolute positions given in parameter positions
    positions needs to be between 0 and 50,
    axes that should be moved are given in parameter axes
    '''
    if len(axes) != len(positions):
        print('number of axes and positions must be the same!')
        return None
    
    c_id = convert_id(controller_id)
    sz_axes = get_szAxes(axes)
    c_double_array = create_double_array(len(axes), positions)
    success = c848.C848_MOV(c_id, sz_axes, c_double_array)
    
    return bool(success)

In [14]:
# Niektóre funkcje dll dla C848 muszą wywoływać się na pojedyńczej osi na raz\
# Inaczej działają tylko na pierwszej podanej osi
# Jest to jakiś błąd

def check_reference_status(controller_id, axes='xyz'):
    '''
    function check if given axes have reference
    returns dictionary with axes as keys and boolean values
    '''
    c_id = convert_id(controller_id)
    status = {}
    for c in axes:
        axis = get_szAxes(c)
        bool_array = create_bool_array(size=1)
        check = c848.C848_IsReferenceOK(c_id, axis, bool_array)
        
        if check != 1:
            print('something went terribly wrong')
            return
        
        status[c] = bool_array[0]
    return status


def is_referencing(controller_id, axes):
    '''
    checks if any axis is referencing in the moment
    returns boolean value - True if any axis is in the middle of referencing
    '''
    c_id = convert_id(controller_id)
    status = {}
    for c in axes:
        axis = get_szAxes(c)
        bool_array = create_bool_array(size=1)
        check = c848.C848_IsReferencing(c_id, axis, bool_array)
        
        if check != 1:
            print('something went terribly wrong')
            return
        
        status[c] = bool_array[0]
    return any(status.values())

def reference_axes(controller_id, axes='xyz'):  
    '''
    moves given axes to reference points and sets references
    '''
    c_id = convert_id(controller_id)
    sz_axes = get_szAxes(axes)
    success = c848.C848_REF(c_id, sz_axes)
    return bool(success)

def init_axes(controller_id, axes='xyz'):    
    '''
    trzeba doczytac - kasuje refenrecje na pewno
    '''
    c_id = convert_id(controller_id)
    sz_axes = get_szAxes(axes)
    success = c848.C848_INI(c_id, sz_axes)
    return bool(success)

In [15]:
def emergency_stop(controller_id):
    '''
    emergency stop of movements for all axes
    '''
    c_id = convert_id(controller_id)
    return c848.C848_EmergencyStop(c_id)

In [16]:
def simple_stop(controller_id):
    '''
    stops movement of all axes
    '''
    c_id = convert_id(controller_id)
    return c848.C848_STP(c_id)

In [17]:
def check_on_target(controller_id, axes='xyz'):
    '''
    function check if given axes is on target
    returns dictionary with axes as keys and boolean values
    '''
    c_id = convert_id(controller_id)
    status = {}
    for c in axes:
        axis = get_szAxes(c)
        bool_array = create_bool_array(size=1)
        check = c848.C848_qONT(c_id, axis, bool_array)
        
        if check != 1:
            print('something went terribly wrong')
            return
        
        status[c] = bool_array[0]
    return status

def check_referencing_mode(controller_id, axes='xyz'):
    '''
    function check if given axes are in referencing mode
    returns dictionary with axes as keys and boolean values
    '''
    c_id = convert_id(controller_id)
    status = {}
    for c in axes:
        axis = get_szAxes(c)
        bool_array = create_bool_array(size=1)
        check = c848.C848_qRON(c_id, axis, bool_array)
        
        if check != 1:
            print('something went terribly wrong')
            return
        
        status[c] = bool_array[0]
    return status

def set_referencing_mode(controller_id, axes='xyz', on=True):
    '''
    function sets axes to referencing mode or turns it of
    '''
    c_id = convert_id(controller_id)
    for c in axes:
        axis = get_szAxes(c)
        bool_array = create_bool_array(size=1, values=on)
        check = c848.C848_RON(c_id, axis, bool_array)
    return bool(check)

In [18]:
def set_abs_positions(controller_id, axes='xyz', positions = None):
    '''
    set absolut positions for not referenced axes
    
    referencing mode has to be turned off (function set_referencing_mode)
    '''
    if positions == None:
        return None
    
    if len(axes) != len(positions):
        print('number of axes and positions must be the same!')
        return None
    
    c_id = convert_id(controller_id)
    sz_axes = get_szAxes(axes)
    c_double_array = create_double_array(len(axes), positions)
    success = c848.C848_POS(c_id, sz_axes, c_double_array)
    
    return bool(success)

In [20]:
c848 = load_drivers()
print(c848)

controller_id = conncect_to_controller(c848)
print('controller_id:', controller_id)
print('is connected:', is_connected(controller_id))

<CDLL 'C848_DLL.dll', handle 10000000 at 0x3fd63d0>
controller_id: 0
is connected: True


In [29]:
move_axes_to_abs(controller_id, axes='xyz', positions=[25, 30., 27.0])

True

In [85]:
simple_stop(controller_id)

1

In [26]:
reference_axes(controller_id, axes='xyz')

True

In [23]:
is_referencing(controller_id, axes='xyz')

False

In [27]:
get_axes_positions(controller_id, axes='xyz')

[25.0, 25.0, 25.0]

In [34]:
close_connection(controller_id)

In [49]:
check_reference_status(controller_id, axes='xyz')

{'x': True, 'y': True, 'z': True}

In [ ]:
# BOOL C848_IsMoving (const const int ID, char *const szAxes, BOOL * pbValarray)

In [33]:
positions = get_axes_positions(controller_id, 'xyz')
positions

[25.0, 25.0, 25.0]

In [20]:
def save_positions(controller_id):
    '''
    saves positions of axes in a config file
    '''
    axes = 'xyz'
    positions = get_axes_positions(controller_id, axes)
    with open('positions.txt', 'w') as file:
        for ax, position in zip(axes, positions):
            file.write('{}: {}\n'.format(ax, position))

In [21]:
def read_positions():
    '''
    read positions of axes from a config file
    '''
    position_dict = {}
    with open('positions.txt', 'r') as file:
        for line in file:
            ax, position = line.split(':')
            position_dict[ax.strip()] = float(position.strip())
    return position_dict

In [22]:
def split_axes_positions(position_dict):
    '''
    reads dictionary with posiotions and converts it to tuple with axes string and list of positions
    '''
    axes = []
    positions = []
    for ax, position in position_dict.items():
        axes.append(ax)
        positions.append(position)

    axes = ''.join(axes)
    return axes, positions

In [23]:
def set_abs_positions_from_file(controller_id):
    '''
    function turns off referencing mode and sets axes absolute positions read from conf file.
    '''
    set_referencing_mode(controller_id, on=False)
    position_dict = read_positions()
   # print(position_dict)
    axes, positions = split_axes_positions(position_dict)
   # print(axes, positions)
    return set_abs_positions(controller_id, axes=axes, positions=positions)

In [66]:
set_abs_positions_from_file(controller_id)

True

In [64]:
save_positions(controller_id)

In [68]:
check_referencing_mode(controller_id, 'xyz')

{'x': False, 'y': False, 'z': False}

In [82]:
set_referencing_mode(controller_id, on=False)
set_abs_positions(controller_id)

In [67]:
check_referencing_mode(controller_id, 'xyz')

{'x': False, 'y': False, 'z': False}

In [20]:
bool_array_3 = create_bool_array(size=3)
bool_array_3[0:3]

[False, False, False]

In [35]:
check_on_target(controller_id)

{'x': True, 'y': True, 'z': True}

In [88]:
check_reference_status(controller_id, 'xyz')

{'x': True, 'y': True, 'z': True}

In [85]:
init_axes(controller_id)

True

In [56]:
check_reference_status(controller_id, 'xyz')

{'x': False, 'y': False, 'z': False}

In [57]:
reference_axes(controller_id, 'xyz')

True

In [60]:
is_referencing(controller_id, 'xyz')

False

In [61]:
check_reference_status(controller_id, 'xyz')

{'x': True, 'y': True, 'z': True}

In [99]:
set_referencing_mode(controller_id, on=False)

True

In [100]:
check_referencing_mode(controller_id, 'xyz')

{'x': False, 'y': False, 'z': False}

In [49]:
emergency_stop(controller_id)

1

In [648]:
c848.C848_MNL(int_id, char_axes)

1

In [264]:
temp[0]

30.0

In [174]:
a = d_point[:len(axes)]
a

[37.420001, 28.174001, 0.0]

In [176]:
a[3]

IndexError: list index out of range

In [158]:
char = get_szAxes('xYz')
char.value

b'ABC'

In [162]:
i = convert_id(controller_id)
i

c_long(0)

In [21]:
b = create_bool_array(3)
b[0]

True

In [ ]:
c848.C848_MNL(ctypes.c_int(id), char)

In [18]:
c848 = load_drivers()
print(c848)

controller_id = conncect_to_controller(c848)
print('controller_id:', controller_id)
print('is connected:', is_connected(controller_id))

<CDLL 'C848_DLL.dll', handle 10000000 at 0x3f26eb0>
controller_id: None


TypeError: an integer is required (got type NoneType)

In [88]:
close_connection(controller_id)

In [ ]:
A = x
B = y
C = z

In [ ]:
c848.C848_DEM(ctypes.c_int(id))

In [35]:
c848.C848_STP(ctypes.c_int(id))

1

In [54]:
# creates array of 3 doubles and pointer to it
arr = (ctypes.c_double * 3)(*range(3))
d_point = ctypes.cast(arr, ctypes.POINTER(ctypes.c_double))

In [55]:
# empty string (pointer to char)
ctypes.c_char_p(b'')

c_char_p(4566184)

c_char_p(9219240)

In [57]:
c848.C848_qMOV(ctypes.c_int(controller_id), ctypes.c_char_p(b''), d_point)

1

In [58]:
d_point[0], d_point[1], d_point[2], 

(0.0, 50.0, 0.0)

In [33]:
d_point[0], d_point[1], d_point[2], 

(25.0, 25.0, 25.10318)

In [37]:
d_point[0], d_point[1], d_point[2], 

(25.264224, 26.681667, 25.883215)

In [338]:
char = ctypes.c_char_p('C'.encode('utf-8'))

In [339]:
char.value

b'C'

In [335]:
init_ax = c848.C848_INI

In [336]:
init_ax.argtypes = [ctypes.c_int, ctypes.c_char_p]

In [337]:
init_ax(3, 'C'.encode('utf-8'))

0

In [340]:
c848.C848_MPL(ctypes.c_int(controller_id), char)

1

In [346]:
b = (ctypes.c_bool *1)(1)
b = ctypes.cast(b, ctypes.POINTER(ctypes.c_bool))
successful = c848.C848_IsReferencing(ctypes.c_int(controller_id), char, b)
if successful:
    print('IsReferencing:', b[0])

IsReferencing: False


In [187]:
b = (ctypes.c_double *10)(*range(10))
b = ctypes.cast(b, ctypes.POINTER(ctypes.c_double))
successful = c848.C848_qNLM((ctypes.c_int(id), char, b))
if successful:
    print('IsReferencing:', b[0])

ArgumentError: argument 1: <class 'TypeError'>: Don't know how to convert parameter 1

In [178]:
b = (ctypes.c_bool *3)(1, 1, 1)
b = ctypes.cast(b, ctypes.POINTER(ctypes.c_bool))

In [175]:
b[2]

True

In [ ]:
A = x
B = y
C = z

In [202]:
cont_id = ctypes.c_int(id)
ax = ctypes.c_char_p('A'.encode('utf-8'))
arr = (ctypes.c_double * 1)(25)
positions = ctypes.cast(arr, ctypes.POINTER(ctypes.c_double))

In [199]:
d_point[2]

0.0

In [332]:
# BOOL C848_MOV (const int ID, char *const szAxes, double * pdValarray)
# Move szAxes to absolute position
cont_id = ctypes.c_int(controller_id)
ax = ctypes.c_char_p('C'.encode('utf-8'))
arr = (ctypes.c_double * 2)(10.1)
positions = ctypes.cast(arr, ctypes.POINTER(ctypes.c_double))

In [333]:
c848.C848_MOV(cont_id, ax, positions)

0

In [10]:
# BOOL C848_qREF (const int ID, char *const szAxes, BOOL * pbValarray)
# check if given axis has reference
c_id = ctypes.c_int(controller_id)
ax = ctypes.c_char_p('ABC'.encode('utf-8'))
b = (ctypes.c_bool *1)(0)
b = ctypes.cast(b, ctypes.POINTER(ctypes.c_bool))
b[0]

False

In [11]:
c848.C848_qREF(c_id, ax, b), b[0]

(1, True)

In [68]:
# BOOL C848_qSAI (const int ID, char * axes, const int maxlen)
# check what axes are connected:
c_id = ctypes.c_int(controller_id)
ax = ctypes.c_char_p('ZZZ'.encode('utf-8'))
buff = ctypes.c_int(4)
ax.value, buff

(b'ZZZ', c_long(4))

In [69]:
c848.C848_qSAI(c_id, ax, buff), ax.value

(1, b'ABC')

In [123]:
# BOOL C848_qPOS (const int ID, char *const szAxes, double * pdValarray)
# Get the positions of szAxes. You can call C848_HasPosChanged() (p.17) to find out if this call is necessary.
c_id = ctypes.c_int(controller_id)
ax = ctypes.c_char_p('ABC'.encode('utf-8'))
arr = (ctypes.c_double * 3)(1.111, 1.111, 1.111)#, 4, 11)
d_point = ctypes.cast(arr, ctypes.POINTER(ctypes.c_double))


c_id, ax.value, d_point[0]

(c_long(0), b'ABC', 1.111)

In [125]:
if c848.C848_qPOS(c_id, ax, d_point):
    print(d_point[0], d_point[1], d_point[2])
else:
    print('cos nie dziala')

37.420001 28.174001 0.0
